## Fine-tuning a model on a text classification task
This notebook will show to fine-tune one of the 🤗 Transformers model to a text classification task of the [GLUE Benchmark](https://gluebenchmark.com/).
<br><br>
The **GLUE Benchmark** is a group of nine classification tasks on sentences or pairs of sentences which are:

- **CoLA** (Corpus of Linguistic Acceptability) Determine if a sentence is grammatically correct or not.is a dataset containing sentences labeled grammatically correct or not.
- **MNLI** (Multi-Genre Natural Language Inference) Determine if a sentence entails, contradicts or is unrelated to a given hypothesis. (This dataset has two versions, one with the validation and test set coming from the same distribution, another called mismatched where the validation and test use out-of-domain data.)
- **MRPC** (Microsoft Research Paraphrase Corpus) Determine if two sentences are paraphrases from one another or not.
- **QNLI** (Question-answering Natural Language Inference) Determine if the answer to a question is in the second sentence or not. (This dataset is built from the SQuAD dataset.)
- **QQP** (Quora Question Pairs2) Determine if two questions are semantically equivalent or not.
- **RTE** (Recognizing Textual Entailment) Determine if a sentence entails a given hypothesis or not.
- **SST-2** (Stanford Sentiment Treebank) Determine if the sentence has a positive or negative sentiment.
- **STS-B** (Semantic Textual Similarity Benchmark) Determine the similarity of two sentences with a score from 1 to 5.
- **WNLI** (Winograd Natural Language Inference) Determine if a sentence with an anonymous pronoun and a sentence with this pronoun replaced are entailed or not. (This dataset is built from the Winograd Schema Challenge dataset.)

In [1]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.1MB 5.9MB/s 
     |████████████████████████████████| 901kB 38.5MB/s 
     |████████████████████████████████| 3.3MB 36.9MB/s 
     |████████████████████████████████| 194kB 4.1MB/s 
     |████████████████████████████████| 112kB 5.7MB/s 
     |████████████████████████████████| 245kB 6.8MB/s 


In [2]:
import datasets
from datasets import load_dataset, load_metric

import random
import pandas as pd
import numpy as np

from IPython.display import display, HTML

import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# datasets : 1.5.0  |  pd : 1.1.5  |  np : 1.19.5  |  transformers : 4.5.1
print(f'datasets : {datasets.__version__}  |  pd : {pd.__version__}  |  np : {np.__version__}  |  transformers : {transformers.__version__}')

datasets : 1.5.0  |  pd : 1.1.5  |  np : 1.19.5  |  transformers : 4.5.1


We will see how to easily load the dataset for each one of those tasks and use the `Trainer` API to fine-tune a model on it. Each task is named by its acronym, with `mnli-mm` standing for the mismatched version of MNLI (so same training set as `mnli` but different validation and test sets):

In [3]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]
task = "cola"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## Loading the dataset & metric
We will use the 🤗 Datasets library to download the data and get the metric we need to use for evaluation (to compare our model to the benchmark). This can be easily done with the functions `load_dataset` and load_metric`.

In [4]:
actual_task = "mnli" if task == "mnli-mm" else task
dataset = load_dataset("glue", actual_task)
metric = load_metric('glue', actual_task)

print('\n>>> actual_task :', actual_task)
print('\n>>> type of metric :', type(metric))
print('\n>>> dataset object :', dataset)
print('\n>>> sample data :', dataset['train'][0])

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/cola/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4. Subsequent calls will reuse this data.




>>> actual_task : cola

>>> type of metric : <class 'datasets_modules.metrics.glue.e4606ab9804a36bcd5a9cebb2cb65bb14b6ac78ee9e6d5981fa679a495dd55de.glue.Glue'>

>>> dataset object : DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

>>> sample data : {'idx': 0, 'label': 1, 'sentence': "Our friends won't buy this analysis, let alone the next one we propose."}


In [5]:
# show random sample of a dataset
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = random.sample(range(len(dataset)), k=num_examples)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(dataset["train"])

,idx,label,sentence
0,8249,unacceptable,Dracula thought that himself was the Prince of Darkness.
1,7311,acceptable,I know that she runs.
2,4276,acceptable,Stephen seemed to be intelligent.
3,8119,acceptable,The very old and extremely wise owl.
4,4004,acceptable,The monkeys kept forgetting their lines.


In [6]:
# compute score with metric
fake_preds = np.random.randint(0, 2, size=(16,))
fake_labels = np.random.randint(0, 2, size=(16,))

fake_preds, fake_labels, metric.compute(predictions=fake_preds, references=fake_labels)

(array([1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0]),
 array([1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1]),
 {'matthews_correlation': -0.1259881576697424})

The metric object only computes the proper metrics associated to the task, which are:

- for **CoLA** : Matthews Correlation Coefficient
- for **MNLI**(matched or mismatched) : Accuracy
- for **MRPC** : Accuracy and F1 score
- for **QNLI** : Accuracy
- for **QQP** : Accuracy and F1 score
- for **RTE** : Accuracy
- for **SST-2** : Accuracy
- for **STS-B** : Pearson Correlation Coefficient and Spearman's_Rank_Correlation_Coefficient
- for **WNLI** : Accuracy